# Introduction

This notebook explores whether the number of users that a user refers and retention is correlated.

# Imports & API Keys

In [20]:
import os
import pandas as pd
import mysql.connector
import plotly.express as px
from plotly import graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from ggplot import *
from scipy.stats.stats import pearsonr
from scipy.stats.stats import pointbiserialr

In [4]:
usr = os.environ.get('sql_user')
pwd = os.environ.get('sql_pwd')
host = os.environ.get('sql_host')
db = os.environ.get('sql_db')

def query(sql):
    
    cnx = mysql.connector.connect(user=usr, 
                              password=pwd,
                              host=host,
                              database=db,
                                 port=3306)
    
    df = pd.read_sql(sql, cnx)
    cnx.close()
    return df

# Queries & Dataframes

Query for user id, D1 referral, D3 referral, D7 referral, D1 modules, D3 modules, D7 modules, D1 retention, D3 retention, D7 retention

In [48]:
full_query ='''
SELECT *
FROM
(
SELECT u.id, num_modules1, num_refer1, num_modules3, num_refer3, num_modules7, num_refer7,
CASE
	WHEN u.id IN 
		(SELECT u.id
		FROM user u
		LEFT JOIN education_completed_module ecm
		ON ecm.user_id = u.id
		WHERE ecm.date_created >= u.date_created + interval 1 day
		AND u.date_created <= now() - interval 7 day
		GROUP BY u.id) THEN 1
    ELSE 0
END AS D1_retained,
CASE
	WHEN u.id IN 
		(SELECT u.id
		FROM user u
		LEFT JOIN education_completed_module ecm
		ON ecm.user_id = u.id
		WHERE ecm.date_created >= u.date_created + interval 3 day
		AND u.date_created <= now() - interval 7 day
		GROUP BY u.id) THEN 1
    ELSE 0
END AS D3_retained,
CASE
	WHEN u.id IN 
		(SELECT u.id
		FROM user u
		LEFT JOIN education_completed_module ecm
		ON ecm.user_id = u.id
		WHERE ecm.date_created >= u.date_created + interval 7 day
		AND u.date_created <= now() - interval 7 day
		GROUP BY u.id) THEN 1
    ELSE 0
END AS D7_retained
FROM user u
LEFT JOIN
(
SELECT u.id, count(DISTINCT ecm.education_module_id) as num_modules1, count(DISTINCT rr.referred_id) as num_refer1
FROM user u
LEFT JOIN education_completed_module ecm
ON ecm.user_id = u.id
LEFT JOIN referral_relationship rr
ON rr.referrer_id = u.id
WHERE (ecm.date_created BETWEEN u.date_created AND u.date_created + interval 1 day OR ecm.date_created IS NULL)
AND (rr.date_executed BETWEEN u.date_created AND u.date_created + interval 1 day OR rr.date_executed IS NULL)
AND u.date_created <= now() - interval 7 day
GROUP BY u.id
) AS 1tab
ON u.id = 1tab.id
LEFT JOIN
(
SELECT u.id, count(DISTINCT ecm.education_module_id) as num_modules3, count(DISTINCT rr.referred_id) as num_refer3
FROM user u
LEFT JOIN education_completed_module ecm
ON ecm.user_id = u.id
LEFT JOIN referral_relationship rr
ON rr.referrer_id = u.id
WHERE (ecm.date_created BETWEEN u.date_created AND u.date_created + interval 3 day OR ecm.date_created IS NULL)
AND (rr.date_executed BETWEEN u.date_created AND u.date_created + interval 3 day OR rr.date_executed IS NULL)
AND u.date_created <= now() - interval 7 day
GROUP BY u.id
) AS 3tab
ON u.id = 3tab.id
LEFT JOIN
(
SELECT u.id, count(DISTINCT ecm.education_module_id) as num_modules7, count(DISTINCT rr.referred_id) as num_refer7
FROM user u
LEFT JOIN education_completed_module ecm
ON ecm.user_id = u.id
LEFT JOIN referral_relationship rr
ON rr.referrer_id = u.id
WHERE (ecm.date_created BETWEEN u.date_created AND u.date_created + interval 7 day OR ecm.date_created IS NULL)
AND (rr.date_executed BETWEEN u.date_created AND u.date_created + interval 7 day OR rr.date_executed IS NULL)
AND u.date_created <= now() - interval 7 day
GROUP BY u.id
) AS 7tab
ON u.id = 7tab.id
GROUP BY u.id
) AS full_tab
WHERE num_modules1 IS NOT NULL
AND num_modules7 <> 0
GROUP BY id
'''

full_df = query(full_query)
full_df

,id,num_modules1,num_refer1,num_modules3,num_refer3,num_modules7,num_refer7,D1_retained,D3_retained,D7_retained
0,124,1,0,1,0,1,0,0,0,0
1,125,1,0,1,0,1,0,0,0,0
2,129,1,0,1,0,1,0,0,0,0
3,130,4,0,4,0,4,0,0,0,0
4,131,5,2,8,2,23,2,1,1,1
...,...,...,...,...,...,...,...,...,...,...
38087,55870,1,0,3,0,5,0,1,1,0
38088,55871,3,0,6,0,6,0,1,0,0
38089,55872,1,0,1,0,1,0,0,0,0
38090,55873,4,0,6,0,6,0,1,0,0


# Data Manipulation

Correlation between num_modules1 and num_refer1:

In [49]:
pearsonr(full_df['num_modules1'], full_df['num_refer1'])

(0.028838591618023733, 1.8077097114576398e-08)

Correlation between num_modules3 and num_refer3:

In [50]:
pearsonr(full_df['num_modules3'], full_df['num_refer3'])

(0.05341490310776635, 1.7677810024141497e-25)

Correlation between num_modules7 and num_refer7:

In [51]:
pearsonr(full_df['num_modules7'], full_df['num_refer7'])

(0.06646446253069745, 1.4706477443759705e-38)

Point-biserial correlation coefficients:

In [39]:
pointbiserialr(full_df['D7_retained'], full_df['num_refer7'])

PointbiserialrResult(correlation=0.024011527708374024, pvalue=0.13306169459695955)

In [40]:
pointbiserialr(full_df['D7_retained'], full_df['num_refer3'])

PointbiserialrResult(correlation=-0.0020215682768223915, pvalue=0.8993763705491901)

In [41]:
pointbiserialr(full_df['D7_retained'], full_df['num_refer1'])

PointbiserialrResult(correlation=-0.008538943073227567, pvalue=0.593258196942434)

In [42]:
pointbiserialr(full_df['D3_retained'], full_df['num_refer7'])

PointbiserialrResult(correlation=0.011937476706086223, pvalue=0.45523528943880304)

In [43]:
pointbiserialr(full_df['D3_retained'], full_df['num_refer3'])

PointbiserialrResult(correlation=-0.019026639038651313, pvalue=0.23395850826792508)

In [44]:
pointbiserialr(full_df['D3_retained'], full_df['num_refer1'])

PointbiserialrResult(correlation=-0.030452622588684433, pvalue=0.05674721957012307)

In [45]:
pointbiserialr(full_df['D1_retained'], full_df['num_refer7'])

PointbiserialrResult(correlation=0.014243010471315891, pvalue=0.3729585630937155)

In [46]:
pointbiserialr(full_df['D1_retained'], full_df['num_refer3'])

PointbiserialrResult(correlation=-0.011426339471207355, pvalue=0.4747693211256349)

In [47]:
pointbiserialr(full_df['D1_retained'], full_df['num_refer1'])

PointbiserialrResult(correlation=-0.0340429940920621, pvalue=0.03317112129809354)

Creating dataframes for number of referrals and proportion of users for D1 retained and unretained users:

In [8]:
d1ret_df = full_df.loc[full_df['D1_retained'] == 'yes']
d1nonret_df = full_df.loc[full_df['D1_retained'] == 'no']

d1ret_df2 = pd.DataFrame(data = d1ret_df['num_refer'].value_counts(normalize=True)).reset_index().sort_values(by = ['index'])
d1nonret_df2 = pd.DataFrame(data = d1nonret_df['num_refer'].value_counts(normalize=True)).reset_index().sort_values(by = ['index'])

d1ret_df3 = d1ret_df2.rename(columns = {'index':'num_refer', 'num_refer':'prop'})
d1nonret_df3 = d1nonret_df2.rename(columns = {'index':'num_refer', 'num_refer':'prop'})

In [10]:
d1ret_df3

,num_refer,prop
0,0,0.807202
1,1,0.112140
2,2,0.034294
3,3,0.016118
4,4,0.008093
5,5,0.004321
6,6,0.003086
8,7,0.001989
7,8,0.002126
10,9,0.001235


Creating dataframes for number of referrals and proportion of users for D3 retained and unretained users:

In [ ]:
d3ret_three_day_df = three_day_df.loc[three_day_df['D3_retained'] == 'yes']
d3nonret_three_day_df = three_day_df.loc[three_day_df['D3_retained'] == 'no']

d3ret_three_day_df2 = pd.DataFrame(data = d3ret_three_day_df['num_refer'].value_counts(normalize=True) * 100).reset_index().sort_values(by = ['index'])
d3nonret_three_day_df2 = pd.DataFrame(data = d3nonret_three_day_df['num_refer'].value_counts(normalize=True) * 100).reset_index().sort_values(by = ['index'])

d3ret_three_day_df3 = d3ret_three_day_df2.rename(columns = {'index':'num_refer', 'num_refer':'prop'})
d3nonret_three_day_df3 = d3nonret_three_day_df2.rename(columns = {'index':'num_refer', 'num_refer':'prop'})

Creating dataframes for number of referrals and proportion of users for D7 retained and unretained users:

In [ ]:
d7ret_three_day_df = three_day_df.loc[three_day_df['D7_retained'] == 'yes']
d7nonret_three_day_df = three_day_df.loc[three_day_df['D7_retained'] == 'no']

d7ret_three_day_df2 = pd.DataFrame(data = d7ret_three_day_df['num_refer'].value_counts(normalize=True) * 100).reset_index().sort_values(by = ['index'])
d7nonret_three_day_df2 = pd.DataFrame(data = d7nonret_three_day_df['num_refer'].value_counts(normalize=True) * 100).reset_index().sort_values(by = ['index'])

d7ret_three_day_df3 = d7ret_three_day_df2.rename(columns = {'index':'num_refer', 'num_refer':'prop'})
d7nonret_three_day_df3 = d7nonret_three_day_df2.rename(columns = {'index':'num_refer', 'num_refer':'prop'})

# Visualizations

In [7]:
fig_ret = go.Figure(data=[
    go.Bar(name='D1 retained', x=d1ret_three_day_df3['num_refer'], y=d1ret_three_day_df3['prop']),
    go.Bar(name='Not D1 retained', x=d1nonret_three_day_df3['num_refer'], y=d1nonret_three_day_df3['prop'])
])


fig_ret.update_layout(barmode = 'group', 
                     title_text = '', 
                     xaxis_title = 'Number of Users Referred',
                     yaxis_title = 'Proportion of Users')

fig_ret.show()

In [21]:
three_day_df.loc[three_day_df['D1_retained'] == 'yes']

,id,num_modules,num_refer,D1_retained,D3_retained,D7_retained
5,131,8,2,yes,yes,yes
8,135,7,0,yes,yes,yes
9,136,23,0,yes,no,no
14,141,2,0,yes,yes,no
15,142,21,2,yes,yes,yes
...,...,...,...,...,...,...
51762,53887,8,0,yes,no,no
51767,53892,13,0,yes,yes,no
51770,53895,1,0,yes,yes,no
51778,53903,10,0,yes,no,no


In [ ]:

fig_ret.update_layout(barmode = 'group', 
                     title_text = 'Daily Objectives Increases Number of Modules Completed', 
                     xaxis_title = 'Number of Completed Modules',
                     yaxis_title = 'Proportion of Users')